In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [3]:
def parse_webpage(link, total_pages):
    i=1
    reviews=[]
    stars=[]
    
    for i in range(total_pages):
        r=requests.get(link + "&pageNumber=" + str(i))
        soup = BeautifulSoup(r.text, "html.parser")
        
        j=1
        results_rev = soup.find_all("span", {'class':"a-size-base review-text review-text-content"})
        temp_reviews = [review.text for review in results_rev]
        for j in range(len(temp_reviews)):
            reviews.append(temp_reviews[j])
            j+=1
        
        
        k=1
        regex=re.compile('a-icon a-icon-star a-star-[12345] review-rating')
        results_stars = soup.find_all("i", {'class':regex})
        temp_stars = [star.text for star in results_stars]
        for k in range(len(temp_stars)):
            stars.append(temp_stars[k])
            k+=1
  
        i+=1
        
    return reviews, stars

In [4]:
def preprocess_data(df, num_col, str_col):
    df[num_col]= df[num_col].apply(lambda x : x.split(" ")[0])
    df[num_col]= pd.to_numeric(df[num_col], errors='coerce')
    df[str_col]= df[str_col].apply(lambda x : x.replace("\n", ""))
    

In [5]:
link="https://www.amazon.in/Power-Your-Subconscious-Mind-Success/product-reviews/8172345666/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

In [10]:
reviews, stars = parse_webpage(link,300)

In [11]:
df=pd.DataFrame(list(zip(reviews,stars)), columns=["reviews", "sentiment"])
df

,reviews,sentiment
0,\nLet me tell you about what the book says: Yo...,4.0 out of 5 stars
1,\nSPOILERS AHEAD PEOPLE!!Murphy's idea is very...,1.0 out of 5 stars
2,\nVery useful book...good for maintaining a ca...,1.0 out of 5 stars
3,\nI have no idea why people buy this one. I bo...,4.0 out of 5 stars
4,"\nThis is my first book, I'm not very fond of ...",5.0 out of 5 stars
...,...,...
2015,\nexcellent book\n,5.0 out of 5 stars
2016,\nGood Book\n,4.0 out of 5 stars
2017,\nIn good condition\n,1.0 out of 5 stars
2018,\nA must read book\n,5.0 out of 5 stars


In [12]:
col_1='sentiment'
col_2='reviews'

preprocess_data(df, col_1, col_2)
df.head()

,reviews,sentiment
0,Let me tell you about what the book says: Your...,4.0
1,SPOILERS AHEAD PEOPLE!!Murphy's idea is very s...,1.0
2,Very useful book...good for maintaining a calm...,1.0
3,I have no idea why people buy this one. I boug...,4.0
4,"This is my first book, I'm not very fond of re...",5.0


In [13]:
df.to_csv('sentiment_data.csv',index=False)